In [40]:
import requests
from bs4 import BeautifulSoup
from tqdm import tqdm
import os
import json
import numpy as np
import time
import random
import json
import pandas as pd
from tika import parser
from bs4.element import Comment
from bs4 import BeautifulSoup

In [45]:
TIKA_HOST = 'http://localhost:9998'

def tag_visible(element):
    if element.parent.name in ['style', 'script', 'head', 'title', 'meta', '[document]']:
        return False
    if isinstance(element, Comment):
        return False
    return True


def text_from_html(body, is_html=True):
    if is_html:
        soup = BeautifulSoup(body, 'html.parser')
    else:
        soup = body
    texts = soup.findAll(text=True)
    visible_texts = filter(tag_visible, texts)
    return '\n'.join(t.strip() for t in visible_texts).strip()


def parse_tika(string, minlen=2,):
    raw_xml = parser.from_buffer(string, TIKA_HOST, xmlContent=True)
    body = BeautifulSoup(raw_xml['content']).find('body')
    t = []
    for b in body:
        try:
            t_ = text_from_html(b, is_html=False)
            if len(t_) > minlen:
                t.append(t_)
        except Exception as e:
            pass
    
    return t

In [4]:
with open('mampu.json') as fopen:
    data = json.load(fopen)
len(data)

12127

In [6]:
directory = 'data-mampu'
!mkdir {directory}
!du -hs {directory}

mkdir: cannot create directory ‘data-mampu’: File exists
5.1G	data-mampu


In [7]:
files = {}
k = 0
for d in data:
    for url in d['file_urls']:
        file, ext = os.path.splitext(url)
        if not len(ext):
            file = f'{k}'
        else:
            file = f'{k}{ext}'
        files[url] = os.path.join(directory, file)
        k += 1

In [50]:
with open('data.gov.my', 'w') as fopen_l:
    for d in tqdm(data):
        file_urls = {}
        for f in d['file_urls']:
            f_f = files[f]
            if not os.path.exists(f_f):
                continue
            try:
                if f_f.endswith('.xlsx'):
                    t = pd.read_excel(f_f).to_csv()
                else:
                    with open(f_f) as fopen:
                        t = fopen.read()
                    if '<html' in t:
                        t = parse_tika(t)
                        t = '\n'.join(t)
                
                file_urls[f] = t
            except:
                pass
        
        d['file_urls'] = file_urls
        fopen_l.write(f'{json.dumps(d)}\n')

  0%|                                        | 29/12127 [00:01<09:53, 20.37it/s]/home/husein/.local/lib/python3.8/site-packages/openpyxl/worksheet/_reader.py:312: UserWarning: Unknown extension is not supported and will be removed
  warn(msg)
  1%|▎                                   | 125/12127 [00:28<11:40:35,  3.50s/it]/home/husein/.local/lib/python3.8/site-packages/openpyxl/worksheet/header_footer.py:48: UserWarning: Cannot parse header or footer so it will be ignored
  warn("""Cannot parse header or footer so it will be ignored""")
  4%|█▍                                     | 433/12127 [00:46<08:12, 23.75it/s]/home/husein/.local/lib/python3.8/site-packages/openpyxl/worksheet/_reader.py:312: UserWarning: Data Validation extension is not supported and will be removed
  warn(msg)
 18%|██████▉                               | 2208/12127 [02:57<05:44, 28.78it/s]/home/husein/.local/lib/python3.8/site-packages/openpyxl/worksheet/_reader.py:312: UserWarning: Conditional Formatting extensio